In [46]:
!pip install pyspark
!pip install findspark

Introdução

Objetivos do Projeto

Este projeto de BigData tem como objetivo usar algoritmos de Machine Learning, recorrendo ao uso do PySpark, pois o mesmo ajuda o processamento de grandes quantidades de dados e é incorpurado com ferramentas para prever resultados importantes em relação a um conjunto de dados.
O Spark é uma ferramenta poderosa que permite o processamento distribuído de dados.
Iremos usar o Spark para ler, limpar e preparar os dados, avaliar e ajustar modelos e, finalmente, fazer previsões precisas e relevantes.
Este projeto é uma excelente oportunidade para aplicar e praticar habilidades de Machine Learning em um ambiente de grande escala e de alta performance.

Introdução ao Dataset

O nosso dataset inclui dados de percentagem de gordura, dados de pressão sanguínea dialóstica e sistólica, alguns exercícios tais como broad jump, abdominais e amplitude no sit and bend;
Temos como objetivo utilizar técnicas Machine Learning para identificar padrões e tendências nos dados e talvez prever a queda de performance com idade ou percentagem de gordura;

Criar uma "SparkSession"

In [47]:
from pyspark.sql import SparkSession
import findspark

findspark.init()

spark = SparkSession.builder.appName("Pyspark ProjetoBD")\
                    .config("spark.memory.offHeap.enabled","true")\
                    .config("spark.memory.offHeap.size","20g")\
                    .getOrCreate()
spark


In [48]:
df = spark.read.csv(r'C:\Users\user\Desktop\ESTB\3 ano\1º semestre\Big Data\Trabalho 2\bodyPerformance.csv',header=True,sep=";")

Visualização dos dados e Análise

In [49]:
print("Numero de Linhas:", df.count())
print("Numero de Colunas:", len(df.columns))
print("Esquema do DataFrame: ")
df.printSchema()

Numero de Linhas: 13393
Numero de Colunas: 12
Esquema do DataFrame: 
root
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- height_cm: string (nullable = true)
 |-- weight_kg: string (nullable = true)
 |-- body fat_%: string (nullable = true)
 |-- diastolic: string (nullable = true)
 |-- systolic: string (nullable = true)
 |-- gripForce: string (nullable = true)
 |-- sit and bend forward_cm: string (nullable = true)
 |-- sit-ups counts: string (nullable = true)
 |-- broad jump_cm: string (nullable = true)
 |-- class: string (nullable = true)



In [50]:
df.show(10)

+----+------+---------+---------+----------+---------+--------+---------+-----------------------+--------------+-------------+-----+
| age|gender|height_cm|weight_kg|body fat_%|diastolic|systolic|gripForce|sit and bend forward_cm|sit-ups counts|broad jump_cm|class|
+----+------+---------+---------+----------+---------+--------+---------+-----------------------+--------------+-------------+-----+
|27.0|     M|    172.3|    75.24|      21.3|     80.0|   130.0|     54.9|                   18.4|          60.0|        217.0|    C|
|25.0|     M|    165.0|     55.8|      15.7|     77.0|   126.0|     36.4|                   16.3|          53.0|        229.0|    A|
|31.0|     M|    179.6|     78.0|      20.1|     92.0|   152.0|     44.8|                   12.0|          49.0|        181.0|    C|
|32.0|     M|    174.5|     71.1|      18.4|     76.0|   147.0|     41.4|                   15.2|          53.0|        219.0|    B|
|28.0|     M|    173.8|     67.7|      17.1|     70.0|   127.0|     4

Definição das variáveis do Dataframe:

age: idade dos participantes deste estudo;
gender: genero dos participantes;
height_cm: altura dos participantes em centímetros;
weight_kg: peso dos participantes em kilogramas;
body fat_%: percentagem de gordura no corpo;
diastolic: pressão sanguínea por minuto em relaxamento;
systolic: prressão sanguínea por minuto em contração;
gripForce: força aplicada pela mão para segurar ou levantar; 
sit and bend forward_cm: amplitude medida em centímetros deste exercício;
sit-ups count: contagem de abdominais;
broad jump_cm: distância em centímetros do salto sem impulso com os dois pés;
class: classificação;

In [51]:
#retirar a variável "class" que não será útil para a nossa análise
df = df.drop("class")

In [52]:
df.count(), len(df.columns)

(13393, 11)

O nosso dataset tem assim 13393 linhas e 11 colunas

Qual a idade mínima e máxima registada?

In [53]:
df.createOrReplaceTempView("data")
spark.sql("SELECT MAX(`age`) FROM data").show()

+--------+
|max(age)|
+--------+
|    64.0|
+--------+



In [54]:
spark.sql("SELECT MIN(`age`) FROM data").show()

+--------+
|min(age)|
+--------+
|    21.0|
+--------+



Verificar as características das nossas variáveis

In [55]:
df.dtypes

[('age', 'string'),
 ('gender', 'string'),
 ('height_cm', 'string'),
 ('weight_kg', 'string'),
 ('body fat_%', 'string'),
 ('diastolic', 'string'),
 ('systolic', 'string'),
 ('gripForce', 'string'),
 ('sit and bend forward_cm', 'string'),
 ('sit-ups counts', 'string'),
 ('broad jump_cm', 'string')]

Como as variáveis escontravam-se todas em tipo String, foi necessário mudar as variáveis.


In [56]:
from pyspark.sql.types import *
from pyspark.sql.types import IntegerType, DoubleType
df = df \
    .withColumn("age", df["age"].cast(IntegerType())) \
    .withColumn("gender", df["gender"].cast(StringType())) \
    .withColumn("height_cm", df["height_cm"].cast(DoubleType())) \
    .withColumn("weight_kg", df["weight_kg"].cast(DoubleType())) \
    .withColumn("body fat_%", df["body fat_%"].cast(DoubleType())) \
    .withColumn("diastolic", df["diastolic"].cast(IntegerType())) \
    .withColumn("systolic", df["systolic"].cast(IntegerType())) \
    .withColumn("gripForce", df["gripForce"].cast(DoubleType())) \
    .withColumn("sit and bend forward_cm", df["sit and bend forward_cm"].cast(DoubleType())) \
    .withColumn("sit-ups counts", df["sit-ups counts"].cast(IntegerType())) \
    .withColumn("broad jump_cm", df["broad jump_cm"].cast(IntegerType())) 
    

In [57]:
df.dtypes

[('age', 'int'),
 ('gender', 'string'),
 ('height_cm', 'double'),
 ('weight_kg', 'double'),
 ('body fat_%', 'double'),
 ('diastolic', 'int'),
 ('systolic', 'int'),
 ('gripForce', 'double'),
 ('sit and bend forward_cm', 'double'),
 ('sit-ups counts', 'int'),
 ('broad jump_cm', 'int')]

In [58]:
df.describe().show()

+-------+-----------------+------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------------+-----------------+------------------+
|summary|              age|gender|        height_cm|         weight_kg|        body fat_%|         diastolic|          systolic|         gripForce|sit and bend forward_cm|   sit-ups counts|     broad jump_cm|
+-------+-----------------+------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------------+-----------------+------------------+
|  count|            13393| 13393|            13393|             13393|             13328|             13393|             13393|             13393|                  13393|            13393|             13393|
|   mean|36.77510639886508|  null|168.5598073620558| 67.44731576196514|23.319833208283306| 78.79675950123199|130.23474949600538|36.963877398641166|      15.20926827

In [59]:
df=df.withColumnRenamed('age','Age')
df=df.withColumnRenamed('gender','Gender')
df=df.withColumnRenamed('height_cm','Height')
df=df.withColumnRenamed('weight_kg','Weight')
df=df.withColumnRenamed('body fat_%','BodyFat')
df=df.withColumnRenamed('diastolic','Dialostic')
df=df.withColumnRenamed('systolic','Systolic')
df=df.withColumnRenamed('gripForce','GripForce')
df=df.withColumnRenamed('sit and bend forward_cm','SitBend')
df=df.withColumnRenamed('sit-ups counts','SitUps')
df=df.withColumnRenamed('broad jump_cm','BroadJump')

Verificar se existem valores nulos 

In [60]:
missing_values = []
for col in df.columns:
    missing_values.append((col, df.filter(df[col].isNull()).count()))
for col, val in missing_values:
    if val == 0:
        print("{} : Nenhum valor nulo".format(col))
    else:
        print("{} : {} missing values".format(col, val))

Age : Nenhum valor nulo
Gender : Nenhum valor nulo
Height : Nenhum valor nulo
Weight : Nenhum valor nulo
BodyFat : 65 missing values
Dialostic : Nenhum valor nulo
Systolic : Nenhum valor nulo
GripForce : Nenhum valor nulo
SitBend : Nenhum valor nulo
SitUps : Nenhum valor nulo
BroadJump : Nenhum valor nulo


In [61]:
df.show()

+---+------+------+------+-------+---------+--------+---------+-------+------+---------+
|Age|Gender|Height|Weight|BodyFat|Dialostic|Systolic|GripForce|SitBend|SitUps|BroadJump|
+---+------+------+------+-------+---------+--------+---------+-------+------+---------+
| 27|     M| 172.3| 75.24|   21.3|       80|     130|     54.9|   18.4|    60|      217|
| 25|     M| 165.0|  55.8|   15.7|       77|     126|     36.4|   16.3|    53|      229|
| 31|     M| 179.6|  78.0|   20.1|       92|     152|     44.8|   12.0|    49|      181|
| 32|     M| 174.5|  71.1|   18.4|       76|     147|     41.4|   15.2|    53|      219|
| 28|     M| 173.8|  67.7|   17.1|       70|     127|     43.5|   27.1|    45|      217|
| 36|     F| 165.4|  55.4|   22.0|       64|     119|     23.8|   21.0|    27|      153|
| 42|     F| 164.5|  63.7|   32.2|       72|     135|     22.7|    0.8|    18|      146|
| 33|     M| 174.9|  77.2|   36.9|       84|     137|     45.9|   12.3|    42|      234|
| 54|     M| 166.8|  

Data Pre-processing

In [62]:
from pyspark.ml.feature import StandardScaler, VectorAssembler
